# Clean data

### Translate subject names to English

In [30]:
subject_translation = {
    "Matemática e suas Tecnologias": "Mathematics",
    "Linguagens, Códigos e suas Tecnologias": "Languages",
    "Ciências Humanas e suas Tecnologias": "Human Sciences",
    "Ciências da Natureza e suas Tecnologias": "Natural Sciences"
}

## Clean Self-Refine

In [33]:
import re
import pandas as pd

def extract_answer_letter(response):
    # Match "Resposta final: C)" or "Resposta final: C"
    match = re.search(r"resposta final\s*[:\-]?\s*([A-E])\s*\)?", response, re.IGNORECASE | re.DOTALL)
    if not match:
        # Try fallback patterns
        match = re.search(r"letra\s+([A-E])\b", response, re.IGNORECASE)
    return match.group(1).upper() if match else None

#### If did not store initial answer already

In [37]:
df_sr = pd.read_csv("results/self-refine_few-shot/self-refine_few-shot_results_2025-03-28_12-33-28.csv")

cleaned_sr_df = pd.DataFrame({
    "id": df_sr["id"],
    "subject": df_sr["subject"].map(subject_translation),
    "ground_truth": df_sr["ground_truth"],
    "predicted": df_sr["predicted"],
    "correct": df_sr["correct"],
    "initial_answer": df_sr["baseline_answer"].apply(extract_answer_letter)
})

cleaned_sr_df.to_csv("results/self-refine_few-shot_clean/self-refine_few-shot_results_clean_2025-03-28_12-33-28.csv", index=False)

cleaned_sr_df.head()

,id,subject,ground_truth,predicted,correct,initial_answer
0,questao_01,Languages,A,A,True,A
1,questao_02,Languages,A,A,True,D
2,questao_03,Languages,C,C,True,C
3,questao_04,Languages,E,E,True,E
4,questao_05,Languages,A,A,True,A


## Clean CoVe

In [40]:
df_cove = pd.read_csv("results/cove_few-shot/cove_few-shot_results_2025-03-28_17-19-55.csv")

cleaned_cove_df = pd.DataFrame({
    "id": df_cove["id"],
    "subject": df_cove["subject"].map(subject_translation),
    "ground_truth": df_cove["ground_truth"],
    "predicted": df_cove["predicted"],
    "correct": df_cove["correct"],
    "initial_answer": df_cove["initial_answer"]
})

cleaned_cove_df.to_csv("results/cove_few-shot_clean/cove_few-shot_results_clean_2025-03-28_17-19-55.csv", index=False)

cleaned_cove_df.head()

,id,subject,ground_truth,predicted,correct,initial_answer
0,questao_01,Languages,A,A,True,A
1,questao_02,Languages,A,D,False,B
2,questao_03,Languages,C,C,True,C
3,questao_04,Languages,E,E,True,E
4,questao_05,Languages,A,A,True,A


## Clean CoT

In [43]:
df_cot = pd.read_csv("results/cot_few-shot/cot_few-shot_results_2025-03-27_13-55-58.csv")


cleaned_cot_df = pd.DataFrame({
    "id": df_cot["id"],
    "subject": df_cot["subject"].map(subject_translation),
    "ground_truth": df_cot["ground_truth"],
    "predicted": df_cot["predicted"],
    "correct": df_cot["correct"],
})

cleaned_cot_df.to_csv("results/cot_few-shot_clean/cot_few-shot_results_clean_2025-03-27_13-55-58.csv", index=False)

cleaned_cot_df.head()

,id,subject,ground_truth,predicted,correct
0,questao_01,Languages,A,A,True
1,questao_02,Languages,A,D,False
2,questao_03,Languages,C,C,True
3,questao_04,Languages,E,E,True
4,questao_05,Languages,A,A,True


## Merge results